# Strip Chart Testing

This notebook is used for testing the strip chart mode on  Auxiliary Telescope.

Craig Lage - 02-Jul-21

In [2]:
import sys
import asyncio
import time
import os
import numpy as np
from astropy.time import Time, TimeDelta
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [3]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [4]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [6]:
#Start classes
domain = salobj.Domain()
latiss = LATISS(domain)
await latiss.start_task

atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.06 sec
Read 2 history items for RemoteEvent(ATCamera, 0, appliedSettingsMatchStart)
Read 1 history items for RemoteEvent(ATCamera, 0, calibrationDetailedState)
Read 27 history items for RemoteEvent(ATCamera, 0, ccsCommandState)
Read 6 history items for RemoteEvent(ATCamera, 0, endOfImageTelemetry)
Read 6 history items for RemoteEvent(ATCamera, 0, endReadout)
Read 3 history items for RemoteEvent(ATCamera, 0, endShutterClose)
Read 3 history items for RemoteEvent(ATCamera, 0, endShutterOpen)
Read 3 history items for RemoteEvent(ATCamera, 0, focalPlaneHardwareIdSettingsApplied)
Read 3 history items for RemoteEvent(ATCamera, 0, focalPlaneSummaryInfo)
Read 100 history items for RemoteEvent(ATCamera, 0, heartbeat)
Read 25 history items for RemoteEvent(ATCamera, 0, imageReadinessDetailedState)
Read 6 history items for RemoteEve

[None, None, None, None]

In [ ]:
# enable components
# No need - all enabled
#await latiss.enable({"atspectrograph": "current"})

In [7]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000007])

In [8]:
# Take one more
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000008])

Step one - go in and out of calibration.\
This succeeded - I could see the camera substate change.

In [9]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [10]:
await asyncio.sleep(2.0)

In [11]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [12]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000009])

Step one repeated - go in and out of calibration.\
This succeeded - I could see the camera substate change.

In [13]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [14]:
await asyncio.sleep(2.0)

In [15]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [16]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000010])

Step 1.5:
In and out of calibration with a clear.\
Succeeded

In [17]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [18]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [19]:
await asyncio.sleep(2.0)

In [20]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [21]:
# Take a bias to make sure everything is working
# Succeeded
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000011])

Step 2:
The following commands should take a calibration image\
Try this first without the discardRows()\
This worked.  The shutter opened and closed, and successfully took image 0012.

In [22]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [23]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [24]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [25]:
print(keyValueMap)

groupId: 2021-09-30T21:03:02.821,imageType: ENGTEST


In [26]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)

True

In [27]:
await latiss.rem.atcamera.cmd_startImage.start()

In [28]:
await asyncio.sleep(2.0)

In [29]:
await latiss.rem.atcamera.cmd_endImage.start()
# Worked this time

In [30]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [31]:
# Take a bias to make sure everything is working
# Succeeded.
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000013])

Step 2A:
The following commands should take a calibration image\
Try this first without the discardRows()\
Trying again to see if it enters CalibrationIntegrating state and if it stays there.\
It did enter the CalibrationIntegrating state, but then it times out while I was checking the substate.\
But it successfully took image 14 and a bias(15) afterwards, so the camera is still alive.

In [32]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [33]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [34]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [35]:
print(keyValueMap)

groupId: 2021-09-30T21:06:28.613,imageType: ENGTEST


In [36]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=10.0)

True

In [37]:
await latiss.rem.atcamera.cmd_startImage.start()

In [38]:
await asyncio.sleep(2.0)

In [39]:
await latiss.rem.atcamera.cmd_endImage.start()
# This failed because it timed out while I was checking the substate status.

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=717841637, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid calibration state State{CalibrationState = ENABLED}')

In [40]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [41]:
# Take a bias to make sure everything is still working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000015])

Step 2B:
The following commands should take a calibration image\
Try this first without the discardRows()\
Trying again to see if it enters CalibrationIntegrating state and if it stays there.\
Setting the timeout to 20 seconds.\
Successfully took image 16 and a bias(17) afterwards.

In [42]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [43]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [44]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [45]:
print(keyValueMap)

groupId: 2021-09-30T21:10:50.826,imageType: ENGTEST


In [46]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=20.0)

True

In [47]:
await latiss.rem.atcamera.cmd_startImage.start()

In [48]:
await asyncio.sleep(2.0)

In [49]:
await latiss.rem.atcamera.cmd_endImage.start()
# Worked this time

In [50]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [51]:
# Take a bias to make sure everything is still working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000017])

Step 3:
The following commands should take a calibration image\
With the discardRows().\
This worked, and successfully took image 18 and a subsequent bias(19).\
But image 18 looks normal.  Did it work?  I need to study the docs some more\
to see what the image should look like.

In [52]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [53]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [54]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [55]:
print(keyValueMap)

groupId: 2021-09-30T21:13:52.824,imageType: ENGTEST


In [56]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=20.0)

True

In [57]:
await latiss.rem.atcamera.cmd_startImage.start()

In [58]:
await asyncio.sleep(2.0)

In [59]:
latiss.rem.atcamera.cmd_discardRows.set(nRows=500)

True

In [60]:
await latiss.rem.atcamera.cmd_discardRows.start()

In [61]:
await asyncio.sleep(1.0)

In [62]:
await latiss.rem.atcamera.cmd_endImage.start()

In [63]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [64]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000019])

logMessage DDS read queue is filling: 11 of 100 elements


Step 3A:
The following commands should take a calibration image\
With the discardRows().\
Trying again with discardRows repeated four times.  This should make\
a visible change to the image.\
No change to image(20) seen.  Successful bias afterwards(21).

In [65]:
await latiss.rem.atcamera.cmd_enableCalibration.set_start()

In [66]:
await latiss.rem.atcamera.cmd_clear.set_start(nClears=2)

In [67]:
keyValueMap = f"groupId: {Time.now().isot},imageType: ENGTEST" 

In [68]:
print(keyValueMap)

groupId: 2021-09-30T21:26:16.361,imageType: ENGTEST


In [69]:
latiss.rem.atcamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=20.0)

True

In [70]:
await latiss.rem.atcamera.cmd_startImage.start()

In [71]:
await asyncio.sleep(2.0)

In [72]:
latiss.rem.atcamera.cmd_discardRows.set(nRows=500)
for i in range(4):
    await latiss.rem.atcamera.cmd_discardRows.start()
    await asyncio.sleep(0.1)
await latiss.rem.atcamera.cmd_endImage.start()

In [73]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [74]:
# Take a bias to make sure everything is working
await latiss.take_bias(1)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001


array([2021093000021])

In [75]:
# Putting everything back in standby.
await latiss.standby()

[atcamera]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atspectrograph]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atheaderservice]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[atarchiver]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


Stuff below is just in case it is needed.

In [ ]:
await latiss.rem.atcamera.cmd_disable.set_start()

In [ ]:
latiss.rem.atcamera.cmd_endImage.set_start()

In [ ]:
await latiss.rem.atcamera.cmd_disableCalibration.start()

In [ ]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.ENABLED)

In [ ]:
test = await latiss.rem.atcamera.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
print(test)

In [ ]:
print(test)

In [ ]:
await latiss.rem.atcamera.cmd_disable?

In [ ]:
await latiss.rem.atcamera.cmd_disable

In [ ]:
await latiss.rem.atcamera.cmd_disable.set_start()

In [ ]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY)

In [ ]:
await latiss.rem.atspectrograph.cmd_enable.set_start({"atspectrograph": "current"})

In [ ]:
await latiss.rem.atspectrograph.enable({"atspectrograph": "current"})

In [ ]:
await latiss.enable({"atspectrograph": "current"})

In [ ]:
await latiss.rem.atcamera.cmd_initImage.set_start(deltaT=2.0)

In [ ]:
await latiss.rem.atcamera.cmd_abort.start()

In [ ]:
await latiss.rem.atcamera.cmd_stop.start()

In [ ]:
await latiss.rem.atcamera.cmd_stop

In [ ]:
await latiss.rem.atcamera.cmd_endImage.set?

In [ ]:
await latiss.rem.atcamera.cmd_endImage.set

In [ ]:
await latiss.rem.atcamera.cmd_endImage